# Consumer Opt. with PV

In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating project at `C:\Users\ESchr\workspace\uni\ATIS3\Single_Project\ATIES3_single_project_ES`


In [2]:
using JuMP, LinearAlgebra, StatsPlots, Statistics, CSV, DataFrames, XLSX, CPLEX, AxisArrays

## Input handling

In [11]:
input_path = "input\\"
xl = XLSX.readxlsx(input_path * "input.xlsx") 
data = Dict()

for s in XLSX.sheetnames(xl)
    data[s] = DataFrame(XLSX.gettable(xl[s]; infer_eltypes=true)...)
end

#Rename column from gams export

## Create Axis-Arrays

In [27]:
#Sets
W_PP = data["pool_price_scenarios"][:, :Scenario]
W_NAM = data["Non_anticipativity_matrix"][:, :Scenario]
B = data["bilateral_contract_data"][:, :Contract]
D = data["electricity_demand"][:, :Demand]
T = ["t1", "t2", "t3", "t4"]

# Risk
β = 0.0
α = 0.95
println("Sets created")

Sets created


In [28]:
Dpool_price_scenarios = Dict(c => AxisArray(data["pool_price_scenarios"][:, c], w_pp=W_PP) for c in names(data["pool_price_scenarios"]));
Dpool_price_scenarios["t1"]["w1"]

Dbilateral_contract_data = Dict(c => AxisArray(data["bilateral_contract_data"][:, c], b=B) for c in names(data["bilateral_contract_data"]));
Dbilateral_contract_data["price"]["b1"]

Delectricity_demand = Dict(c => AxisArray(data["electricity_demand"][:, c], d=D) for c in names(data["electricity_demand"]));
Delectricity_demand["t1"]["d1"]

Dnon_anticipativity_matrix = Dict(c => AxisArray(data["Non_anticipativity_matrix"][:, c], w_nam=W_NAM) for c in names(data["Non_anticipativity_matrix"]));
Dnon_anticipativity_matrix["k1"]["w1"]
println("Axis Arrays created")

Axis Arrays created


In [31]:
#Mapping of bilateral contracts active in period T
Bt = Dict()
for t in T
    temp = String[]
    for b in B
        if(Dbilateral_contract_data[t][b]==1)
            push!(temp, b)
        end
    end
    Bt[t] = temp
end 

#Mapping of decisions on bilateral contracts made in stage K
Kb = Dict()
Kb["k1"] = Bt["t1"]
Kb["k2"] = Bt["t2"]
Kb["k3"] = Bt["t3"]
Kb["k4"] = Bt["t4"]

3-element Vector{String}:
 "b1"
 "b2"
 "b6"

In [54]:
co_pv = Model(CPLEX.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX

In [55]:
@variables(co_pv, begin
        z #Objective function value
        PB[B,W_PP] #Power purchased from bilateral contracts
        s[B,W_PP], Bin #Selection of bilateral contracts
        #EG[N,T,W] >= 0 #Energy produced by the self-production unit
        EP[T,W_PP] #Energy traded in the pool       
        
        η[W_PP] >= 0 #Auxiliary variable used to calculate the CVaR
        ζ #Auxiliary variable used to calculate the CVaR
    end)

(z, 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["b1", "b2", "b3", "b4", "b5", "b6"]
    Dimension 2, ["w1", "w2", "w3", "w4", "w5", "w6", "w7", "w8", "w9", "w10", "w11", "w12", "w13", "w14", "w15", "w16"]
And data, a 6×16 Matrix{VariableRef}:
 PB[b1,w1]  PB[b1,w2]  PB[b1,w3]  …  PB[b1,w14]  PB[b1,w15]  PB[b1,w16]
 PB[b2,w1]  PB[b2,w2]  PB[b2,w3]     PB[b2,w14]  PB[b2,w15]  PB[b2,w16]
 PB[b3,w1]  PB[b3,w2]  PB[b3,w3]     PB[b3,w14]  PB[b3,w15]  PB[b3,w16]
 PB[b4,w1]  PB[b4,w2]  PB[b4,w3]     PB[b4,w14]  PB[b4,w15]  PB[b4,w16]
 PB[b5,w1]  PB[b5,w2]  PB[b5,w3]     PB[b5,w14]  PB[b5,w15]  PB[b5,w16]
 PB[b6,w1]  PB[b6,w2]  PB[b6,w3]  …  PB[b6,w14]  PB[b6,w15]  PB[b6,w16], 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["b1", "b2", "b3", "b4", "b5", "b6"]
    Dimension 2, ["w1", "w2", "w3", "w4", "w5", "w6", "w7", "w8", "w9", "w10", "w11", "w12", "w13", "w14", "w15", "w16"]
And data, a 6×16 Matrix{VariableRef}:
 s[b1,w

In [56]:
@constraints(co_pv, begin
        LimitContractMin[b in B, w in W_PP], PB[b,w] >= Dbilateral_contract_data["min_power"][b]*s[b,w]
        LimitContractMax[b in B, w in W_PP], PB[b,w] <= Dbilateral_contract_data["max_power"][b]*s[b,w]
        

        EnergyBalance[t in T, w in W_PP], EP[t,w] + sum(PB[b,w]*Delectricity_demand[t]["d1"] for b in Bt[t]) == Delectricity_demand[t]["d1"]
        #Arbitrage[t in T, w in W], sum(EG[n,t,w] for n in N) >= -EP[t,w]
        #Non_Anticipativity $(KB(B,K) and (A(W,K)=1) and (ord(W) lt card(W))
        #Non_Anticipativity[b in B, w in W, k in K], PB[b,w] == PB[b, w+1]
        #CVaRconstr[w in W], sum(sum( λ_B[b,t,w]*PB[b,w]*d[t] for b in B #=B$Bt(B,T)=#) + λ_P[w,t]*EP[t,w] + sum(SG[n]*EG[n,t,w] for n in N) for t in T)- ζ - η[w] == 0        
        
        
        #attetion hard coded
        #SelfProduction1[nl in 1:1, t in T, w in W], EG[nl,t,w] <= EGT[nl]
        #SelfProduction[nl in 2:NL, t in T, w in W], EG[nl,t,w] <= EGT[nl] - EGT[nl-1]
    end)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, ["b1", "b2", "b3", "b4", "b5", "b6"]
    Dimension 2, ["w1", "w2", "w3", "w4", "w5", "w6", "w7", "w8", "w9", "w10", "w11", "w12", "w13", "w14", "w15", "w16"]
And data, a 6×16 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 LimitContractMin[b1,w1] : PB[b1,w1] - 20 s[b1,w1] >= 0.0  …  LimitContractMin[b1,w16] : PB[b1,w16] - 20 s[b1,w16] >= 0.0
 LimitContractMin[b2,w1] : PB[b2,w1] - 20 s[b2,w1] >= 0.0     LimitContractMin[b2,w16] : PB[b2,w16] - 20 s[b2,w16] >= 0.0
 LimitContractMin[b3,w1] : PB[b3,w1] - 20 s[b3,w1] >= 0.0     LimitContractMin[b3,w16] : PB[b3,w16] - 20 s[b3,w16] >= 0.0
 LimitContractMin[b4,w1] : PB[b4,w1] - 20 s[b4,w1] >= 0.0     LimitCo

In [ ]:
@constraints(co_pv, begin
        LimitContractMin[b in B, w in W], PB[b,w] >= Dbilateral_contract_data["min_power"][b]*s[b,w]
        LimitContractMax[b in B, w in W], PB[b,w] <= Dbilateral_contract_data["max_power"][b]*s[B,W]
        

        EnergyBalance[t in T, w in W], sum(EG[t,w] for n in N) + EP[t,w] + sum(PB[b,w]*d[t] for b in B)#=$Bt(B,T)=# == Delectricity_demand["t_1"]["d1"]#-EPC[t]
        #Arbitrage[t in T, w in W], sum(EG[n,t,w] for n in N) >= -EP[t,w]
        #Non_Anticipativity $(KB(B,K) and (A(W,K)=1) and (ord(W) lt card(W))
        #Non_Anticipativity[b in B, w in W, k in K], PB[b,w] == PB[b, w+1]
        #CVaRconstr[w in W], sum(sum( λ_B[b,t,w]*PB[b,w]*d[t] for b in B #=B$Bt(B,T)=#) + λ_P[w,t]*EP[t,w] + sum(SG[n]*EG[n,t,w] for n in N) for t in T)- ζ - η[w] == 0        
        
        
        #attetion hard coded
        #SelfProduction1[nl in 1:1, t in T, w in W], EG[nl,t,w] <= EGT[nl]
        #SelfProduction[nl in 2:NL, t in T, w in W], EG[nl,t,w] <= EGT[nl] - EGT[nl-1]
    end)

In [57]:
co_pv

A JuMP Model
Feasibility problem with:
Variables: 274
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 64 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 96 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 96 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 16 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 96 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX
Names registered in the model: EP, EnergyBalance, LimitContractMax, LimitContractMin, PB, s, z, ζ, η